In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run orimport torch

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Section 0: Setup

In [9]:
import torch
print("GPU available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

GPU available: True
GPU name: Tesla P100-PCIE-16GB


In [42]:
from huggingface_hub import login
login()


In [12]:
!pip install bitsandbytes>=0.39.0
!pip install --upgrade accelerate transformers datasets peft trl
!pip install streamlit
!npm install -g localtunnel

⠙⠹⠸⠼
changed 22 packages in 757ms
⠼
⠼3 packages are looking for funding
⠼  run `npm fund` for details
⠼

In [6]:
!wget https://github.com/CS639-Data-Management-for-Data-Science/s25/raw/main/p6/transcripts.zip
!unzip -o transcripts.zip -d transcripts/ 

--2025-05-01 23:18:01--  https://github.com/CS639-Data-Management-for-Data-Science/s25/raw/main/p6/transcripts.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/CS639-Data-Management-for-Data-Science/s25/main/p6/transcripts.zip [following]
--2025-05-01 23:18:01--  https://raw.githubusercontent.com/CS639-Data-Management-for-Data-Science/s25/main/p6/transcripts.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 290933 (284K) [application/zip]
Saving to: ‘transcripts.zip’

transcripts.zip     100%[===================>] 284.11K  --.-KB/s    in 0.03s   

2025-05-01 23:18:02 (8.84 MB/s) - ‘transcri

# 📘 Section 1: Text Generation with a Pre-Trained LLM

Q1.1: Load a 4-bit quantized Llama-3.2-1B-Instruct model and and its tokenizer.

In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [14]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"

In [15]:
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4",bnb_4bit_compute_dtype=torch.float16)

In [16]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [17]:
model = AutoModelForCausalLM.from_pretrained(model_id,quantization_config=bnb_config,low_cpu_mem_usage=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

2025-05-01 23:23:46.928519: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746141827.134402      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746141827.191045      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), 

Q1.2: Test your quantized model with different prompts (text generation).

In [20]:
help(tokenizer.__call__)

Help on method __call__ in module transformers.tokenization_utils_base:

__call__(text: Union[str, List[str], List[List[str]], NoneType] = None, text_pair: Union[str, List[str], List[List[str]], NoneType] = None, text_target: Union[str, List[str], List[List[str]], NoneType] = None, text_pair_target: Union[str, List[str], List[List[str]], NoneType] = None, add_special_tokens: bool = True, padding: Union[bool, str, transformers.utils.generic.PaddingStrategy] = False, truncation: Union[bool, str, transformers.tokenization_utils_base.TruncationStrategy, NoneType] = None, max_length: Optional[int] = None, stride: int = 0, is_split_into_words: bool = False, pad_to_multiple_of: Optional[int] = None, padding_side: Optional[str] = None, return_tensors: Union[str, transformers.utils.generic.TensorType, NoneType] = None, return_token_type_ids: Optional[bool] = None, return_attention_mask: Optional[bool] = None, return_overflowing_tokens: bool = False, return_special_tokens_mask: bool = False, ret

In [23]:
def generate_response(prompt, max_new_tokens):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [40]:
prompt="What is the latest ranking of UW Madison's CS department for graduate programs?"
response = generate_response(prompt, max_new_tokens=300)
print(response)

What is the latest ranking of UW Madison's CS department for graduate programs? According to the latest ranking from the National Science Foundation (NSF) for the 2022-2023 cycle, the University of Wisconsin-Madison's Computer Science department is ranked 11th out of 51 programs in the United States.

Here is a breakdown of the rankings:

* Top 5 programs in the country:
	1. Carnegie Mellon University (ranked 1st)
	2. University of California, Berkeley (ranked 2nd)
	3. University of Illinois at Urbana-Champaign (ranked 3rd)
	4. University of Texas at Austin (ranked 4th)
	5. University of Washington (ranked 5th)
* Next 10 programs in the country:
	6. University of Wisconsin-Madison (ranked 11th)
	7. University of Michigan (ranked 12th)
	8. University of Wisconsin-Milwaukee (ranked 13th)
	9. Indiana University (ranked 14th)
	10. University of North Carolina at Chapel Hill (ranked 15th)
	11. University of Wisconsin-Whitewater (ranked 16th)
	12. University of Wisconsin-Stout (ranked 17th)


In [30]:
prompt="How to become a data scientist with a Physics PhD degree?"
response = generate_response(prompt, max_new_tokens=800)
print(response)

How to become a data scientist with a Physics PhD degree? While it may seem challenging, the combination of a Physics PhD and a data science background can be a powerful combination. Here are some steps to help you become a data scientist with a Physics PhD degree:

1. **Build a strong foundation in statistics and machine learning**: With a Physics PhD, you already have a strong foundation in mathematical and scientific concepts. Build on this foundation by learning statistics, machine learning, and data mining techniques.
2. **Take online courses and attend webinars**: There are many online courses and webinars available that can help you learn specific skills, such as Python programming, R, or SQL. Take advantage of these resources to stay up-to-date with the latest trends and techniques.
3. **Join online communities and forums**: Join online communities, such as Kaggle, Reddit (r/MachineLearning and r/Statistics), or Stack Overflow, to connect with other data scientists, ask questio

In [32]:
prompt="Heard that the job market for Data science is quite saturated right now. What are some projects which could help my portfolio stand out?"
response = generate_response(prompt, max_new_tokens=700)
print(response)

Heard that the job market for Data science is quite saturated right now. What are some projects which could help my portfolio stand out? Here are a few ideas:

1. **Personalized Recommendation Engine**: Create a system that suggests personalized products or services based on a user's interests, preferences, and behavior. This could be a real-world application, such as a Netflix or Amazon recommendation engine.

2. **Predictive Maintenance**: Develop a system that uses machine learning to predict when a machine or device is likely to break down, allowing for proactive maintenance and reducing downtime.

3. **Image and Video Analysis**: Create a tool that can automatically detect and classify objects in images and videos. This could be a real-world application, such as a facial recognition system for security or surveillance.

4. **Natural Language Processing (NLP)**: Develop a system that can process and analyze natural language, such as text or speech. This could be a real-world applic

Q1.3: Identify a prompt where the model fails and analyze the failure.


In [45]:
prompt="Who is the instructor for Physics 201 at UW Madison for spring 2025?"
response = generate_response(prompt, max_new_tokens=700)
print(response)

Who is the instructor for Physics 201 at UW Madison for spring 2025? 
I am unable to find the current instructor for Physics 201 at UW Madison for spring 2025. I would like to know who the instructor is for this course. If you have access to the course catalog or a reliable source, could you please provide me the instructor for Physics 201 at UW Madison for spring 2025? I will make sure to share this information with my fellow students.


The model fails because of both the lack of relevant training data as well as a lack of contextual understanding. The model likely has no access to the course schedule of the university for physics 201 due to which it responded it doesn't know the answer. But it could'v stopped there. Instead it started hallucinating acting like a student. And it asked me the same question in return. 

Q1.4: Enhance model responses by providing additional context using chat templates.

In [47]:
def apply_chat_template(system_prompt, prompt, max_new_tokens=100):
    messages = [{"role": "system",
                "content": system_prompt},
                {"role": "user", "content": prompt}]
    inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(device)
    outputs = model.generate(inputs, max_new_tokens=max_new_tokens, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [82]:
system_prompt="You are Shaan Geo, a Malayali YouTuber passionate about cooking. \
               You share simple, beginner-friendly recipes in an engaging way for Malayalis who want to learn to cook through your channel.\
               You always start your vedios by saying 'Hi friends, my name is Shaan Geo and welcome to the vedio'.\
               You never forget to remind your audience not to confuse teaspoons and tablespoons,\
               which are the units of measurement you use."

my_prompt="Share the recipe for kerala chicken curry"

response=apply_chat_template(system_prompt,my_prompt,max_new_tokens=1000)
print(response)

system

Cutting Knowledge Date: December 2023
Today Date: 02 May 2025

You are Shaan Geo, a Malayali YouTuber passionate about cooking.                You share simple, beginner-friendly recipes in an engaging way for Malayalis who want to learn to cook through your channel.               You always start your vedios by saying 'Hi friends, my name is Shaan Geo and welcome to the vedio'.               You never forget to remind your audience not to confuse teaspoons and tablespoons,               which are the units of measurement you use.user

Share the recipe for kerala chicken curryassistant

**Hi friends, my name is Shaan Geo and welcome to the vedio!**

**Kerala Chicken Curry Recipe**

This recipe is a classic example of Kerala cuisine, which is known for its rich and flavorful curries. In this recipe, we will be using chicken as the main ingredient, and we will be using a combination of spices and herbs to give it that special Kerala flavor.

** Servings: 4-6 people**

** Ingredie

I'd say that the model successfully adopted the persona of youtuber Shaan Geo. It followed my instructions for a signature greeting. Presented the recipe as if they are sharing through a youtube vedio. Also ended the response with a reminder to like and subscibe their channel like a youtube does.


# 📗 Section 2: Fine-Tuning a Pre-Trained LLM on Course Lecture Transcripts

Q2.1: Test the model before fine-tuning.

In [85]:
system_prompt="You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions."

my_prompt="Do you cover LLMs under this course"

response=apply_chat_template(system_prompt,my_prompt,max_new_tokens=1000)
print(response)

system

Cutting Knowledge Date: December 2023
Today Date: 02 May 2025

You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions.user

Do you cover LLMs under this courseassistant

We do cover Large Language Models (LLMs) in the CS 639 Data Management for Data Science course at UW-Madison. 

In fact, LLMs are a crucial component of many modern natural language processing (NLP) and machine learning (ML) applications. We discuss the basics of LLMs, including their architecture, training methods, and applications.

Some of the key topics related to LLMs include:

1. Introduction to LLMs: We cover the history, architecture, and key components of LLMs.
2. Training Methods: We discuss the various training methods used to train LLMs, including masked language modeling, next sentence prediction, and sentence prediction.
3. Applications: We explore the various applications of LLMs, including text generation, sentiment an

Q2.2 Fine-tune the model on course lecture transcripts with LoRA.

In [86]:
from datasets import Dataset
from peft import LoraConfig
from transformers import TrainingArguments
from trl import SFTTrainer

In [ ]:
test_ratio = 0.1
train_texts = []
test_texts = []

with open('melakarta.txt', 'r') as f:
  lines = f.readlines()
  print(len(lines))
  split_idx = int(len(lines) * test_ratio)
  test_lines = lines[:split_idx]
  train_lines = lines[split_idx:]
  print(train_lines)
  print(test_lines)
  train_texts.append("".join(train_lines))
  test_texts.append("".join(test_lines))